In [98]:
import pandas as pd
import re

In [99]:
train_df = pd.read_csv("../../data/train_df_translate_del.csv")
test_df = pd.read_csv("../../data/test_df_translate_del.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

In [100]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

In [101]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   분류      23405 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [102]:
import pandas as pd
import numpy as np

def predict_labels(probabilities_csv, train_df, threshold, output_csv):
    # 확률 데이터 불러오기
    probabilities_df = pd.read_csv(probabilities_csv)

    # 가장 높은 확률과 두 번째로 높은 확률을 가진 클래스 예측
    top_two_indices = np.argsort(probabilities_df.values, axis=1)[:, -2:]
    top_two_probs = np.sort(probabilities_df.values, axis=1)[:, -2:]

    # 레이블 인코더 생성 (이전에 사용한 레이블 인코더와 동일해야 함)
    label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
    inverse_label_encoder = {v: k for k, v in label_encoder.items()}

    # 예측 결과를 저장할 리스트 초기화
    predicted_class_indices = []

    for i in range(len(probabilities_df)):
        # 가장 높은 확률을 가진 클래스 인덱스
        top_class_index = top_two_indices[i, 1]

        # 해당 클래스가 레이블 인코딩에서 1번으로 저장되는 범주인지 확인
        if top_class_index == 1 and top_two_probs[i, 1] < threshold:
            # 임계값을 넘지 않으면 두 번째로 높은 확률의 클래스를 선택
            predicted_class_indices.append(top_two_indices[i, 0])
        else:
            # 그렇지 않으면 가장 높은 확률의 클래스를 선택
            predicted_class_indices.append(top_class_index)

    # 예측 결과를 레이블로 변환
    predicted_labels = [inverse_label_encoder[idx] for idx in predicted_class_indices]

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame({
        'ID': [f'TEST_{i:05d}' for i in range(len(predicted_labels))],  # ID를 TEST_00000 형식으로 생성
        '분류': predicted_labels
    })

    # 결과를 CSV 파일로 저장
    result_df.to_csv(output_csv, index=False, encoding='UTF-8-sig')

    # '분류' 열에서 '지역'에 해당하는 데이터의 비율 계산
    total_count = len(result_df)
    region_count = result_df['분류'].value_counts().get('지역', 0)
    region_ratio = region_count / total_count

    print(f"예측 데이터셋 중 '지역'에 해당하는 데이터의 비율: {region_ratio:.2%}")

In [105]:
# '분류' 열에서 '지역'에 해당하는 데이터의 비율 계산
total_count = len(train_df)
region_count = train_df['분류'].value_counts().get('지역', 0)
region_ratio = region_count / total_count

print(f"train data 중 '지역'에 해당하는 데이터의 비율: {region_ratio:.2%}")

train data 중 '지역'에 해당하는 데이터의 비율: 49.35%


In [106]:
# 예측데이터 스레스홀드 조절
predict_labels("predictions.csv", train_df, 6.6, "predicted_labels.csv")

예측 데이터셋 중 '지역'에 해당하는 데이터의 비율: 49.38%


In [97]:
predicted_labels = pd.read_csv("predicted_labels.csv")

predicted_labels['분류'].value_counts()

지역               11558
경제:부동산            1447
사회:사건_사고          1095
경제:반도체            1032
사회:사회일반            691
사회:교육_시험           476
정치:국회_정당           413
스포츠:올림픽_아시안게임      393
사회:의료_건강           365
경제:경제일반            362
경제:취업_창업           343
정치:선거              290
스포츠:골프             287
경제:자동차             273
경제:유통              267
경제:산업_기업           265
사회:장애인             234
IT_과학:모바일          229
문화:전시_공연           217
사회:노동_복지           210
스포츠:축구             184
경제:금융_재테크          171
경제:무역              160
사회:여성              157
사회:환경              152
문화:방송_연예           151
국제                 149
경제:서비스_쇼핑          143
IT_과학:인터넷_SNS      141
정치:행정_자치           137
문화:문화일반            120
IT_과학:IT_과학일반      118
문화:출판               99
정치:청와대              98
IT_과학:과학            94
문화:미술_건축            93
문화:학술_문화재           88
경제:자원               70
문화:요리_여행            69
정치:정치일반             60
문화:종교               59
문화:생활               56
IT_과학:보안            55
사회:미디어     

.